In [ ]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px


spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


app = dash.Dash(__name__)


app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                
                                dcc.Dropdown(id='site-dropdown', options=[{'label': 'All Sites', 'value': 'ALL'},
                                                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                                                        value='ALL',
                                                                        placeholder='Select a Launch Site here',
                                                                        searchable=True),
                                html.Br(),

                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                             
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={2500: '2500',
                                                    5000: '5000',
                                                    7500: '7500'},
                                                value=[min_payload , max_payload]),

                                
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        filtered_df = spacex_df.groupby('Launch Site')['class'].sum().to_frame().reset_index().rename(columns={"class": "nb_success"})
        fig = px.pie(filtered_df, values='nb_success', 
        names='Launch Site', 
        title='Total Success Launches by Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]['class'].value_counts().to_frame().reset_index()
        filtered_df = filtered_df.rename(columns={"index": "class", "class": "count"})
        fig = px.pie(filtered_df, values="count", 
        names="class", 
        title='Total Success Launches for '+ entered_site)
        return fig


@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
               [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site, slider_value):
    if entered_site == 'ALL':
        filtered_df = spacex_df[['Payload Mass (kg)', 'class', 'Booster Version Category']]
        filtered_df = filtered_df[filtered_df['Payload Mass (kg)'].between(slider_value[0], slider_value[1], inclusive=True)]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
        title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]['Payload Mass (kg)', 'class', 'Booster Version Category']
        filtered_df = filtered_df[filtered_df['Payload Mass (kg)'].between(slider_value[0], slider_value[1], inclusive=True)]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
        title='Correlation between Payload and Success for ' + entered_site)
        return fig


if __name__ == '__main__':
    app.run_server()